In [1]:
import pickle
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split, Subset
# import torchvision
import matplotlib.pyplot as plt
import sinabs
import sinabs.activation
import sinabs.layers as sl
from sinabs.from_torch import from_model
import os
import shutil
import samna
from samna.unifirm.timestamp import StopWatch
from sklearn.model_selection import KFold
print(torch.cuda.is_available())
torch.cuda.get_device_name(0)
torch.cuda.empty_cache()

True


In [18]:
path = '/home/xxc/PhD/synsense/Data/mg_tap_9positions/events/taps_trial_23_pose_0_tap_0.pickle'
with open(path, 'rb') as f:
    f = pickle.load(f)
    print(f[0])
    print(len(f))
    print(f[-1])
    print(f[-1].timestamp - f[0].timestamp)
    print(f[:5])

speck2f::event::Spike(layer=13, feature=1, y=53, x=124, timestamp=0)
14988
speck2f::event::Spike(layer=13, feature=0, y=28, x=112, timestamp=1764)
1764
[speck2f::event::Spike(layer=13, feature=1, y=53, x=124, timestamp=0), speck2f::event::Spike(layer=13, feature=1, y=53, x=124, timestamp=0), speck2f::event::Spike(layer=13, feature=1, y=36, x=81, timestamp=1), speck2f::event::Spike(layer=13, feature=1, y=28, x=112, timestamp=1), speck2f::event::Spike(layer=13, feature=0, y=53, x=124, timestamp=4)]


In [14]:
path = '/home/xxc/PhD/synsense/Data/mg_tap_9positions/events/'
for filename in os.listdir(path):
    with open(os.path.join(path, filename), 'rb') as f:
        loaded_data = pickle.load(f)

    base = loaded_data[0].timestamp
    for sp in loaded_data:
        sp.timestamp = (sp.timestamp - base) // 1000

    with open(os.path.join(path, filename), 'wb') as f:
        pickle.dump(loaded_data, f)

In [ ]:
class MyData(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.data_path = os.listdir(self.root_dir)
    
    def __getitem__(self, idx):
        data_name = self.data_path[idx]
        data_item_path = os.path.join(self.root_dir, data_name)
        with open(data_item_path, 'rb') as f:
            data = pickle.load(f)

        data = torch.from_numpy(data).float()
        data = torch.transpose(data, 0, 1)

        if data_name[-6] == '_':
            label = torch.tensor(eval(data_name[-5]), dtype=torch.long)
        else:
            label = torch.tensor(eval(data_name[-6:-4]), dtype=torch.long)
        
        return data, label

    def __len__(self):
        return len(self.data_path)

root_dir_1 = 'F:\Files\PhD/Braille\Data/braille-27letters-sphere/effect-speed/speed-v10-depth2.5/train'
root_dir_2 = 'F:\Files\PhD/Braille\Data/braille-27letters-sphere/effect-speed/speed-v10-depth2.5/test'

train_data = MyData(root_dir_1)
test_data  = MyData(root_dir_2)